# Pre-Processing

In [468]:
import pandas as pd
import numpy as np
import seaborn as sns
import scipy.stats as scp
import plotly.express as px
import matplotlib.pyplot as plt

In [469]:
def cek_null(df):
    col_na = df.isnull().sum().sort_values(ascending=False)
    percent = col_na / len(df) * 100

    missing_data = pd.concat([col_na, percent], axis=1, keys=['Total', 'Percent'])
    print(missing_data[missing_data['Total'] > 0])

def cek_duplikat(df):
    dup = df.duplicated()
    print("Jumlah duplikasi data : " + str(dup.sum()))

Pada kasus pre-processing sebelumnya, ternyata ditemukan masalah di mana informasi Patch 0.0 menghilang dari `final_dataset.csv`

In [470]:
df_final = pd.read_csv('dataset final/final_dataset.csv')
df_final[df_final['Patch'] == 0.0]

,GameID,PlayerName,TeamAbbreviation,Agent,ACS,Kills,Deaths,Assists,PlusMinus,ADR,...,Patch,EventID,EventName,EventStage,Team1ID,Team2ID,Team1,Team2,Team1_MapScore,Team2_MapScore


Padahal di akhir processing untuk `matches.csv` informasi Patch 0.0 masih ada

In [471]:
df_matches_final = pd.read_csv('dataset final/matches_final.csv')
df_matches_final[df_matches_final['Patch'] == 0.0]

,MatchID,Date,Patch,EventID,EventName,EventStage,Team1ID,Team2ID,Team1,Team2,Team1_MapScore,Team2_MapScore
6196,209,2020-06-14 14:00:00,0.0,8,Absolute Masters,Group Stage: Group H,3,63,Ninjas in Pyjamas,InetGamer,2,0
6212,205,2020-06-13 14:30:00,0.0,8,Absolute Masters,Group Stage: Group G,21,101,Prodigy,Paraplegic Buffalos,2,0
6304,178,2020-06-04 14:00:00,0.0,8,Absolute Masters,Group Stage: Group B,86,85,SimpleMinecraftPlayers,Absolute Legends,1,2
6308,413,2020-05-25 08:00:00,0.0,20,Take the Throne,#4: Grand Final,25,20,fish123,StartedFromCS,2,1
6309,174,2020-05-24 14:30:00,0.0,8,Absolute Masters,Group Stage: Group H,102,63,KK VALORANT,InetGamer,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...
6396,10,2020-05-02 15:00:00,0.0,2,Valhalla Invitational,Week 1: Group A,4,11,smoke,Obey Alliance,1,0
6397,12,2020-05-02 15:00:00,0.0,2,Valhalla Invitational,Week 1: Group B,10,8,Highground,JSD,1,0
6398,8,2020-05-02 14:00:00,0.0,2,Valhalla Invitational,Week 1: Group A,4,6,smoke,Orgless,1,0
6399,7,2020-05-02 13:00:00,0.0,2,Valhalla Invitational,Week 1: Group A,4,5,smoke,Last Minute Heroes,1,0


Kami menemukan bahwa hal ini terjadi karena pada pemrosesan `games.csv` terdapat penghilangan row data yang kolomnya berisikan value NaN, seperti yang ditujukan kode berikut, misalnya untuk `MatchID` `209`

In [472]:
df_games = pd.read_csv('dataset/games.csv')
df_games[df_games['MatchID'] == 209]

,No,GameID,MatchID,Map,Team1ID,Team2ID,Team1,Team2,Winner,Team1_Eco,Team1_SemiEco,Team1_SemiBuy,Team1_FullBuy,Team1_TotalRounds,Team2_Eco,Team2_SemiEco,Team2_SemiBuy,Team2_FullBuy,Team2_TotalRounds
12708,15635,351,209,Bind,3,63,Ninjas in Pyjamas,InetGamer,1,NaN,NaN,NaN,NaN,13,NaN,NaN,NaN,NaN,2
12709,15636,352,209,Haven,3,63,Ninjas in Pyjamas,InetGamer,1,NaN,NaN,NaN,NaN,13,NaN,NaN,NaN,NaN,9


In [473]:
df_final_games = pd.read_csv('dataset final/games_final.csv') # dataset games.csv setelah diproses
df_final_games[df_final_games['MatchID'] == 209]

,GameID,MatchID,Map,Team1ID,Team2ID,Team1,Team2,Winner,Team1_TotalRounds,Team2_TotalRounds
12708,351,209,Bind,3,63,Ninjas in Pyjamas,InetGamer,1,13,2
12709,352,209,Haven,3,63,Ninjas in Pyjamas,InetGamer,1,13,9


Akhirnya, kami memutuskan untuk menetapkan 2 dataset final berbeda di mana untuk dataset final yang baru ini akan ditujukan untuk analisis data yang membutuhkan informasi Patch 0.0.

## Tabel Games

In [474]:
df_games

,No,GameID,MatchID,Map,Team1ID,Team2ID,Team1,Team2,Winner,Team1_Eco,Team1_SemiEco,Team1_SemiBuy,Team1_FullBuy,Team1_TotalRounds,Team2_Eco,Team2_SemiEco,Team2_SemiBuy,Team2_FullBuy,Team2_TotalRounds
0,0,60894,62393,Breeze,6903,6020,Booster Seat Gaming,Pho Real,1,2.0,0.0,5.0,13.0,13,4.0,2.0,4.0,10.0,7
1,1,60895,62393,Bind,6903,6020,Booster Seat Gaming,Pho Real,2,3.0,3.0,5.0,4.0,2,2.0,0.0,4.0,9.0,13
2,2,60896,62393,Haven,6903,6020,Booster Seat Gaming,Pho Real,1,2.0,2.0,5.0,12.0,13,2.0,2.0,6.0,11.0,8
3,3,60924,62403,Icebox,7046,7047,Bjor's Kittens,Mugiwara,1,2.0,2.0,4.0,11.0,13,4.0,1.0,2.0,12.0,6
4,4,60925,62403,Haven,7046,7047,Bjor's Kittens,Mugiwara,1,4.0,2.0,3.0,13.0,13,3.0,3.0,4.0,12.0,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12956,15883,6,10,Haven,4,11,smoke,Obey Alliance,2,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0
12957,15884,8,12,Bind,10,8,Highground,JSD,2,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0
12958,15885,4,8,Haven,4,6,smoke,Orgless,2,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0
12959,15886,3,7,Split,4,5,smoke,Last Minute Heroes,2,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0


In [475]:
df_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12961 entries, 0 to 12960
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   No                 12961 non-null  int64  
 1   GameID             12961 non-null  int64  
 2   MatchID            12961 non-null  int64  
 3   Map                12961 non-null  object 
 4   Team1ID            12961 non-null  int64  
 5   Team2ID            12961 non-null  int64  
 6   Team1              12961 non-null  object 
 7   Team2              12961 non-null  object 
 8   Winner             12961 non-null  int64  
 9   Team1_Eco          11927 non-null  float64
 10  Team1_SemiEco      11927 non-null  float64
 11  Team1_SemiBuy      11927 non-null  float64
 12  Team1_FullBuy      11927 non-null  float64
 13  Team1_TotalRounds  12961 non-null  int64  
 14  Team2_Eco          11927 non-null  float64
 15  Team2_SemiEco      11927 non-null  float64
 16  Team2_SemiBuy      119

Setelah mengkaji ulang, terdapat beberapa kolom yang kurang begitu penting untuk analisis data yang melibatkan informasi Patch, yakni:  
`No`, karena hanya sebagai identifier  
`Team1_Eco`, `Team1_SemiEco`, `Team1_SemiBuy`, `Team1_FullBuy`, `Team2_Eco`,`Team2_SemiEco`, `Team2_SemiBuy`, `Team2_FullBuy`, semua kolom ini berisikan informasi ekonomi tim selama games. Informasi ini tidak relevan untuk analisis data yang melibatkan informasi Patch

In [476]:
col_to_drop = ['No', 'Team1_Eco', 'Team1_SemiEco', 'Team1_SemiBuy', 'Team1_FullBuy', 'Team2_Eco', 'Team2_SemiEco', 'Team2_SemiBuy', 'Team2_FullBuy']
df_games_col_drop = df_games.drop(col_to_drop, axis=1)
df_games_col_drop

,GameID,MatchID,Map,Team1ID,Team2ID,Team1,Team2,Winner,Team1_TotalRounds,Team2_TotalRounds
0,60894,62393,Breeze,6903,6020,Booster Seat Gaming,Pho Real,1,13,7
1,60895,62393,Bind,6903,6020,Booster Seat Gaming,Pho Real,2,2,13
2,60896,62393,Haven,6903,6020,Booster Seat Gaming,Pho Real,1,13,8
3,60924,62403,Icebox,7046,7047,Bjor's Kittens,Mugiwara,1,13,6
4,60925,62403,Haven,7046,7047,Bjor's Kittens,Mugiwara,1,13,9
...,...,...,...,...,...,...,...,...,...,...
12956,6,10,Haven,4,11,smoke,Obey Alliance,2,0,0
12957,8,12,Bind,10,8,Highground,JSD,2,0,0
12958,4,8,Haven,4,6,smoke,Orgless,2,0,0
12959,3,7,Split,4,5,smoke,Last Minute Heroes,2,0,0


In [477]:
cek_null(df_games_col_drop)

Empty DataFrame
Columns: [Total, Percent]
Index: []


Perbedaannya, Map dengan value `TBD` ikut terbawa. Informasi ini kurang relevan untuk membantu analisis kami untuk poin eksplorasi, sehingga dianggap sebagai nilai NaN

In [478]:
df_games_col_drop['Map'].unique()

array(['Breeze', 'Bind', 'Haven', 'Icebox', 'Ascent', 'Split', 'Fracture',
       'TBD'], dtype=object)

In [479]:
tbd_count = df_games_col_drop[df_games_col_drop['Map'] == 'TBD'].shape[0] # jumlah row dengan nilai TBD
tbd_count_percent = tbd_count / df_games_col_drop.shape[0] * 100
tbd_count_percent

0.46292724326826634

Karena berjumlah sedikit dapat dibuang

In [480]:
df_games_drop_tbd = df_games_col_drop[df_games_col_drop['Map'] != 'TBD']
df_games_drop_tbd['Map'].unique()

array(['Breeze', 'Bind', 'Haven', 'Icebox', 'Ascent', 'Split', 'Fracture'],
      dtype=object)

## Cek Duplikat

In [481]:
cek_duplikat(df_games_drop_tbd)

Jumlah duplikasi data : 0


In [482]:
df_games_final = df_games_drop_tbd

In [483]:
df_games_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12901 entries, 0 to 12960
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   GameID             12901 non-null  int64 
 1   MatchID            12901 non-null  int64 
 2   Map                12901 non-null  object
 3   Team1ID            12901 non-null  int64 
 4   Team2ID            12901 non-null  int64 
 5   Team1              12901 non-null  object
 6   Team2              12901 non-null  object
 7   Winner             12901 non-null  int64 
 8   Team1_TotalRounds  12901 non-null  int64 
 9   Team2_TotalRounds  12901 non-null  int64 
dtypes: int64(7), object(3)
memory usage: 1.1+ MB


## Import games final ke csv

In [484]:
df_games_final.to_csv('dataset final/games_final_patch0_enb.csv', index=False)

In [485]:
df_scores = pd.read_csv('dataset/scores.csv')
df_scores[df_scores['GameID'] == 351]

,No,GameID,PlayerID,PlayerName,TeamAbbreviation,Agent,ACS,Kills,Deaths,Assists,...,Num_4Ks,Num_5Ks,OnevOne,OnevTwo,OnevThree,OnevFour,OnevFive,Econ,Plants,Defuses
126359,155519,351,10.0,rhyme,NIP,NaN,412.0,22.0,10.0,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
126360,155520,351,6.0,Tenner,NIP,NaN,260.0,16.0,5.0,9.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
126361,155521,351,194.0,Yacine,NIP,NaN,253.0,15.0,9.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
126362,155522,351,278.0,melonhead,NIP,NaN,183.0,11.0,8.0,9.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
126363,155523,351,7.0,bonkar,NIP,NaN,171.0,10.0,6.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
126364,155524,351,228.0,jarod,Inet,NaN,204.0,10.0,15.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
126365,155525,351,226.0,rodeN,Inet,NaN,185.0,9.0,14.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
126366,155526,351,258.0,Ada,Inet,NaN,149.0,7.0,15.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
126367,155527,351,225.0,waneG,Inet,NaN,139.0,7.0,14.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
126368,155528,351,256.0,BouLy,Inet,NaN,131.0,5.0,16.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Tabel Scores

In [486]:
df_scores = pd.read_csv('dataset/scores.csv')
df_scores

,No,GameID,PlayerID,PlayerName,TeamAbbreviation,Agent,ACS,Kills,Deaths,Assists,...,Num_4Ks,Num_5Ks,OnevOne,OnevTwo,OnevThree,OnevFour,OnevFive,Econ,Plants,Defuses
0,0,60894,8419.0,Reduxx,Boos,jett,313.0,24.0,10.0,3.0,...,2.0,0.0,1.0,0.0,0.0,0.0,0.0,74.0,0.0,0.0
1,1,60894,466.0,ChurmZ,Boos,chamber,227.0,16.0,10.0,7.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,67.0,2.0,0.0
2,2,60894,3712.0,diaamond,Boos,sova,226.0,17.0,9.0,8.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,58.0,3.0,0.0
3,3,60894,5099.0,Boltzy,Boos,viper,218.0,17.0,12.0,2.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,48.0,0.0,0.0
4,4,60894,3983.0,Virtyy,Boos,skye,80.0,5.0,13.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128774,157934,13,24.0,Gover,NaN,NaN,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
128775,157935,13,25.0,Jack1,NaN,NaN,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
128776,157936,13,26.0,Rewind,NaN,NaN,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
128777,157937,13,27.0,Woo1y,NaN,NaN,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [487]:
df_scores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128779 entries, 0 to 128778
Data columns (total 29 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   No                128779 non-null  int64  
 1   GameID            128779 non-null  int64  
 2   PlayerID          128692 non-null  float64
 3   PlayerName        128779 non-null  object 
 4   TeamAbbreviation  126763 non-null  object 
 5   Agent             124679 non-null  object 
 6   ACS               128249 non-null  float64
 7   Kills             128289 non-null  float64
 8   Deaths            128289 non-null  float64
 9   Assists           128289 non-null  float64
 10  PlusMinus         127026 non-null  float64
 11  KAST_Percent      2637 non-null    float64
 12  ADR               119904 non-null  float64
 13  HS_Percent        119307 non-null  float64
 14  FirstKills        128249 non-null  float64
 15  FirstDeaths       119314 non-null  float64
 16  FKFD_PlusMinus    11

In [488]:
cek_null(df_scores)

                   Total    Percent
KAST_Percent      126142  97.952306
Defuses            10195   7.916663
OnevThree          10195   7.916663
Num_3Ks            10195   7.916663
Num_2Ks            10195   7.916663
OnevOne            10195   7.916663
Num_5Ks            10195   7.916663
OnevTwo            10195   7.916663
Num_4Ks            10195   7.916663
OnevFour           10195   7.916663
OnevFive           10195   7.916663
Econ               10195   7.916663
Plants             10195   7.916663
HS_Percent          9472   7.355236
FirstDeaths         9465   7.349801
FKFD_PlusMinus      9465   7.349801
ADR                 8875   6.891652
Agent               4100   3.183749
TeamAbbreviation    2016   1.565473
PlusMinus           1753   1.361247
FirstKills           530   0.411558
ACS                  530   0.411558
Assists              490   0.380497
Deaths               490   0.380497
Kills                490   0.380497
PlayerID              87   0.067558


In [489]:
df_scores[df_scores['PlayerName'] == 'Bob']['PlayerID'].unique()

array([ 873., 7854.])

In [490]:
cek_null(df_scores[df_scores['PlayerID'] == 873])

                Total     Percent
KAST_Percent      195  100.000000
Defuses             9    4.615385
Num_3Ks             9    4.615385
FKFD_PlusMinus      9    4.615385
FirstDeaths         9    4.615385
Num_4Ks             9    4.615385
HS_Percent          9    4.615385
ADR                 9    4.615385
Num_5Ks             9    4.615385
OnevOne             9    4.615385
OnevTwo             9    4.615385
OnevThree           9    4.615385
OnevFour            9    4.615385
OnevFive            9    4.615385
Econ                9    4.615385
Plants              9    4.615385
Num_2Ks             9    4.615385
Agent               1    0.512821


In [491]:
# counts = df_scores['PlayerName'].value_counts()

# checklist = []

# for name, freq in counts.items():
#     if name in checklist:
#         continue
#     if freq > 1:
#         data = df_scores[df_scores['PlayerName'] == name]
#         if data['PlayerID'].nunique() > 1:
#             checklist.append(name)

# print(len(checklist))

150

In [492]:
df_scores_col_drop = df_scores.drop(['KAST_Percent', 'PlayerName', 'No'], axis=1)
df_scores_col_drop

,GameID,PlayerID,TeamAbbreviation,Agent,ACS,Kills,Deaths,Assists,PlusMinus,ADR,...,Num_4Ks,Num_5Ks,OnevOne,OnevTwo,OnevThree,OnevFour,OnevFive,Econ,Plants,Defuses
0,60894,8419.0,Boos,jett,313.0,24.0,10.0,3.0,14.0,195.0,...,2.0,0.0,1.0,0.0,0.0,0.0,0.0,74.0,0.0,0.0
1,60894,466.0,Boos,chamber,227.0,16.0,10.0,7.0,6.0,161.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,67.0,2.0,0.0
2,60894,3712.0,Boos,sova,226.0,17.0,9.0,8.0,8.0,148.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,58.0,3.0,0.0
3,60894,5099.0,Boos,viper,218.0,17.0,12.0,2.0,5.0,141.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,48.0,0.0,0.0
4,60894,3983.0,Boos,skye,80.0,5.0,13.0,3.0,-8.0,55.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128774,13,24.0,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
128775,13,25.0,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
128776,13,26.0,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
128777,13,27.0,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [493]:
match_patch0 = df_matches_final[df_matches_final['Patch'] == 0.0]

matchID = []

for label, value in match_patch0['MatchID'].items():
    matchID.append(value)

In [494]:
gameID = []

for id in matchID:
    game_patch0 = df_games_final[df_games_final['MatchID'] == id]
    for game in game_patch0['GameID'].unique():
        if game not in gameID:
            gameID.append(game)

print(f'gameID size = {len(gameID)}')

gameID size = 88


In [495]:
patch0_agent = []

for id in gameID:
    data_patch0 = df_scores[df_scores['GameID'] == id]
    agent = data_patch0['Agent'].unique()
    if len(agent) == 1:
        if pd.isnull(agent[0]) == False:
            patch0_agent.append(id)
    else:
        for item in agent:
            if pd.isnull(item) == False:
                if id not in patch0_agent:
                    patch0_agent.append(id)

print(f'Patch 0.0 Data with NaN agent = {len(patch0_agent)}')

Patch 0.0 Data with NaN agent = 11


In [496]:
print(patch0_agent)

[339, 340, 273, 274, 275, 255, 256, 257, 219, 220, 221]


In [497]:
df_scores_null_drop = df_scores_col_drop.dropna(subset=['HS_Percent', 'Plants', 'PlayerID', 'FirstKills', 'ACS', 'Assists', 'Deaths', 'Kills', 'Agent', 'TeamAbbreviation'])
cek_null(df_scores_null_drop)

Empty DataFrame
Columns: [Total, Percent]
Index: []


In [517]:
for id in patch0_agent:
    data = df_scores_null_drop[df_scores_null_drop['GameID'] == 339]
    if data.shape[0] != 0:
        print(data)

In [498]:
df_scores_final = df_scores_null_drop

# Menggabungkan Tabel

In [499]:
print(df_scores_final.columns)
print(df_games_final.columns)
print(df_matches_final.columns)

Index(['GameID', 'PlayerID', 'TeamAbbreviation', 'Agent', 'ACS', 'Kills',
       'Deaths', 'Assists', 'PlusMinus', 'ADR', 'HS_Percent', 'FirstKills',
       'FirstDeaths', 'FKFD_PlusMinus', 'Num_2Ks', 'Num_3Ks', 'Num_4Ks',
       'Num_5Ks', 'OnevOne', 'OnevTwo', 'OnevThree', 'OnevFour', 'OnevFive',
       'Econ', 'Plants', 'Defuses'],
      dtype='object')
Index(['GameID', 'MatchID', 'Map', 'Team1ID', 'Team2ID', 'Team1', 'Team2',
       'Winner', 'Team1_TotalRounds', 'Team2_TotalRounds'],
      dtype='object')
Index(['MatchID', 'Date', 'Patch', 'EventID', 'EventName', 'EventStage',
       'Team1ID', 'Team2ID', 'Team1', 'Team2', 'Team1_MapScore',
       'Team2_MapScore'],
      dtype='object')


In [500]:
print(f"Jumlah baris scores: {df_scores_final.count().unique()}")
print(f"Jumlah baris scores: {df_games_final.count().unique()}")
print(f"Jumlah baris scores: {df_matches_final.count().unique()}")

Jumlah baris scores: [116868]
Jumlah baris scores: [12901]
Jumlah baris scores: [6401]


## Menggabungkan Matches dengan Games

In [501]:
df_matches_games = df_games_final.merge(df_matches_final, left_on='MatchID', right_on='MatchID')

In [502]:
df_matches_games.columns

Index(['GameID', 'MatchID', 'Map', 'Team1ID_x', 'Team2ID_x', 'Team1_x',
       'Team2_x', 'Winner', 'Team1_TotalRounds', 'Team2_TotalRounds', 'Date',
       'Patch', 'EventID', 'EventName', 'EventStage', 'Team1ID_y', 'Team2ID_y',
       'Team1_y', 'Team2_y', 'Team1_MapScore', 'Team2_MapScore'],
      dtype='object')

In [503]:
df_matches_games['Patch'].value_counts()

Patch
3.0    4748
2.0    4364
1.0    3701
0.0      88
Name: count, dtype: int64

In [515]:
df_matches_games[df_matches_games['GameID'] == 339]

,GameID,MatchID,Map,Team1ID_x,Team2ID_x,Team1_x,Team2_x,Winner,Team1_TotalRounds,Team2_TotalRounds,...,Patch,EventID,EventName,EventStage,Team1ID_y,Team2ID_y,Team1_y,Team2_y,Team1_MapScore,Team2_MapScore
12711,339,205,Bind,21,101,Prodigy,Paraplegic Buffalos,1,13,2,...,0.0,8,Absolute Masters,Group Stage: Group G,21,101,Prodigy,Paraplegic Buffalos,2,0


## Menggabungkan Tabel secara keseluruhan

In [504]:
df_final = df_scores_final.merge(df_matches_games, left_on='GameID', right_on='GameID')
df_final.columns

Index(['GameID', 'PlayerID', 'TeamAbbreviation', 'Agent', 'ACS', 'Kills',
       'Deaths', 'Assists', 'PlusMinus', 'ADR', 'HS_Percent', 'FirstKills',
       'FirstDeaths', 'FKFD_PlusMinus', 'Num_2Ks', 'Num_3Ks', 'Num_4Ks',
       'Num_5Ks', 'OnevOne', 'OnevTwo', 'OnevThree', 'OnevFour', 'OnevFive',
       'Econ', 'Plants', 'Defuses', 'MatchID', 'Map', 'Team1ID_x', 'Team2ID_x',
       'Team1_x', 'Team2_x', 'Winner', 'Team1_TotalRounds',
       'Team2_TotalRounds', 'Date', 'Patch', 'EventID', 'EventName',
       'EventStage', 'Team1ID_y', 'Team2ID_y', 'Team1_y', 'Team2_y',
       'Team1_MapScore', 'Team2_MapScore'],
      dtype='object')

Ternyata terdapat tabel dengan kolom nama yang sama namun atributnya berbeda, sehingga memunculkan beberapa kolom dengan tambahan `_x` dan `_y`

In [505]:
mask = ['Team1ID_x', 'Team1ID_y', 'Team2ID_x', 'Team2ID_y','Team1_x', 'Team1_y', 'Team2_x', 'Team2_y']
temp = df_final[mask]
temp[temp['Team1_x'] != temp['Team1_y']]

,Team1ID_x,Team1ID_y,Team2ID_x,Team2ID_y,Team1_x,Team1_y,Team2_x,Team2_y
3513,6144,6144,6478,6478,The Mafia,Next-Gen(The Mafia),ex-Serenity Black,ex-Serenity Black
3514,6144,6144,6478,6478,The Mafia,Next-Gen(The Mafia),ex-Serenity Black,ex-Serenity Black
3515,6144,6144,6478,6478,The Mafia,Next-Gen(The Mafia),ex-Serenity Black,ex-Serenity Black
3516,6144,6144,6478,6478,The Mafia,Next-Gen(The Mafia),ex-Serenity Black,ex-Serenity Black
3517,6144,6144,6478,6478,The Mafia,Next-Gen(The Mafia),ex-Serenity Black,ex-Serenity Black
...,...,...,...,...,...,...,...,...
115410,547,547,711,711,ex-Giants Gaming,Fire in the hole(ex-Giants Gaming),The Mandalorian,The Mandalorian
115411,547,547,711,711,ex-Giants Gaming,Fire in the hole(ex-Giants Gaming),The Mandalorian,The Mandalorian
115412,547,547,711,711,ex-Giants Gaming,Fire in the hole(ex-Giants Gaming),The Mandalorian,The Mandalorian
115413,547,547,711,711,ex-Giants Gaming,Fire in the hole(ex-Giants Gaming),The Mandalorian,The Mandalorian


In [506]:
mask = ['Team1ID_x', 'Team1ID_y', 'Team2ID_x', 'Team2ID_y','Team1_x', 'Team1_y', 'Team2_x', 'Team2_y']
temp = df_final[mask]
temp[temp['Team2_x'].str.len() > temp['Team2_y'].str.len()]

,Team1ID_x,Team1ID_y,Team2ID_x,Team2ID_y,Team1_x,Team1_y,Team2_x,Team2_y


Setelah mengamati data tersebut, menurut kami, data `_y` memiliki informasi yang lebih lengkap ketimbang `_x` sehingga kami memilih yang `_y`

In [507]:
col_to_drop = ['Team1ID_x', 'Team2ID_x', 'Team1_x', 'Team2_x']
df_final.drop(col_to_drop, axis=1, inplace=True)
df_final.rename(columns={'Team1ID_y': 'Team1ID', 'Team2ID_y': 'Team2ID', 'Team1_y': 'Team1', 'Team2_y': 'Team2'}, inplace=True)


In [508]:
df_final.columns

Index(['GameID', 'PlayerID', 'TeamAbbreviation', 'Agent', 'ACS', 'Kills',
       'Deaths', 'Assists', 'PlusMinus', 'ADR', 'HS_Percent', 'FirstKills',
       'FirstDeaths', 'FKFD_PlusMinus', 'Num_2Ks', 'Num_3Ks', 'Num_4Ks',
       'Num_5Ks', 'OnevOne', 'OnevTwo', 'OnevThree', 'OnevFour', 'OnevFive',
       'Econ', 'Plants', 'Defuses', 'MatchID', 'Map', 'Winner',
       'Team1_TotalRounds', 'Team2_TotalRounds', 'Date', 'Patch', 'EventID',
       'EventName', 'EventStage', 'Team1ID', 'Team2ID', 'Team1', 'Team2',
       'Team1_MapScore', 'Team2_MapScore'],
      dtype='object')

# Hasil final

In [509]:
cek_null(df_final)

Empty DataFrame
Columns: [Total, Percent]
Index: []


In [510]:
cek_duplikat(df_final)

Jumlah duplikasi data : 0


In [511]:
df_final

,GameID,PlayerID,TeamAbbreviation,Agent,ACS,Kills,Deaths,Assists,PlusMinus,ADR,...,Patch,EventID,EventName,EventStage,Team1ID,Team2ID,Team1,Team2,Team1_MapScore,Team2_MapScore
0,60894,8419.0,Boos,jett,313.0,24.0,10.0,3.0,14.0,195.0,...,3.0,826,Nerd Street Gamers Winter Championship - Regio...,Group Stage: Decider (A),6903,6020,Booster Seat Gaming,Pho Real,2,1
1,60894,466.0,Boos,chamber,227.0,16.0,10.0,7.0,6.0,161.0,...,3.0,826,Nerd Street Gamers Winter Championship - Regio...,Group Stage: Decider (A),6903,6020,Booster Seat Gaming,Pho Real,2,1
2,60894,3712.0,Boos,sova,226.0,17.0,9.0,8.0,8.0,148.0,...,3.0,826,Nerd Street Gamers Winter Championship - Regio...,Group Stage: Decider (A),6903,6020,Booster Seat Gaming,Pho Real,2,1
3,60894,5099.0,Boos,viper,218.0,17.0,12.0,2.0,5.0,141.0,...,3.0,826,Nerd Street Gamers Winter Championship - Regio...,Group Stage: Decider (A),6903,6020,Booster Seat Gaming,Pho Real,2,1
4,60894,3983.0,Boos,skye,80.0,5.0,13.0,3.0,-8.0,55.0,...,3.0,826,Nerd Street Gamers Winter Championship - Regio...,Group Stage: Decider (A),6903,6020,Booster Seat Gaming,Pho Real,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116808,2309,2181.0,GRG,omen,227.0,13.0,12.0,1.0,1.0,124.0,...,1.0,68,Mandatory.gg Cup #2,Tournament: Round of 128,545,560,ISOL-BULIL,GRANIT GAMING,1,0
116809,2309,2179.0,GRG,phoenix,110.0,4.0,13.0,4.0,-9.0,84.0,...,1.0,68,Mandatory.gg Cup #2,Tournament: Round of 128,545,560,ISOL-BULIL,GRANIT GAMING,1,0
116810,2309,2178.0,GRG,jett,98.0,4.0,15.0,0.0,-11.0,56.0,...,1.0,68,Mandatory.gg Cup #2,Tournament: Round of 128,545,560,ISOL-BULIL,GRANIT GAMING,1,0
116811,2309,2180.0,GRG,sova,91.0,3.0,14.0,4.0,-11.0,64.0,...,1.0,68,Mandatory.gg Cup #2,Tournament: Round of 128,545,560,ISOL-BULIL,GRANIT GAMING,1,0


Kita cek kembali informasi Patch 0.0

In [512]:
df_final[df_final['Patch'] == 0.0]

,GameID,PlayerID,TeamAbbreviation,Agent,ACS,Kills,Deaths,Assists,PlusMinus,ADR,...,Patch,EventID,EventName,EventStage,Team1ID,Team2ID,Team1,Team2,Team1_MapScore,Team2_MapScore


## Export dataset hasil gabungan ke csv

In [513]:
df_final.to_csv('dataset final/final_dataset_patch0_enb.csv', index=False)

In [514]:
dataset_final = pd.read_csv('dataset final/final_dataset_patch0_enb.csv')
dataset_final['Patch'].value_counts()

Patch
3.0    46316
2.0    42723
1.0    27774
Name: count, dtype: int64